# Behavior-trait analysis, demographic data

In [ ]:
import numpy as np
import scipy as sp
from scipy import stats
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

plt.rcParams.update({'font.size': 16})

In [ ]:
# setting default fontsizes for plots

s=20# CHANGE FONTSIZE HERE

plt.rc('font', size=s) #controls default text size
plt.rc('axes', titlesize=s) #fontsize of the title
plt.rc('axes', labelsize=s) #fontsize of the x and y labels
plt.rc('xtick', labelsize=s) #fontsize of the x tick labels
plt.rc('ytick', labelsize=s) #fontsize of the y tick labels
plt.rc('legend', fontsize=s) #fontsize of the legend
plt.rcParams['savefig.facecolor']='white'
#import matplotlib as mpl
#mpl.rcParams['font.weight']= 'normal'



Read in restricted behavioral data:

In [ ]:
res_behav_data = pd.read_csv('../data/RESTRICTED_esfinn_11_21_2021_19_19_35.csv')
res_behav_data.set_index("Subject", inplace=True)
res_behav_data.index = res_behav_data.index.map(str)
print(res_behav_data.shape)
res_behav_data.head()

Read in unrestricted behavioral data:

In [ ]:
unres_behav_data = pd.read_csv('../data/unrestricted_esfinn_11_21_2021_19_19_13.csv')
unres_behav_data.set_index("Subject", inplace=True)
unres_behav_data.index = unres_behav_data.index.map(str)
print(unres_behav_data.shape)
unres_behav_data.head()

In [ ]:
[i for i in res_behav_data.columns]

In [ ]:
unres_behav_data.shape,res_behav_data.shape

Combine restricted and unrestricted behavioral data into a single dataframe:

In [ ]:
behav_data = pd.concat([res_behav_data, unres_behav_data], axis=1)
print(behav_data.shape)
behav_data.head()

In [ ]:
behav_data['ASR_Intn_T'].describe()

# Behavior-trait analysis

Read in task response data, join into single dataframe, then join this with the larger dataframe:

In [ ]:
behavioral_data_drive = r'/Users/f0053cz/Dropbox (Dartmouth College)/postdoc_Dartmouth/HCP/BehaviorAnalyses/Documented scripts/data/data_for_paper'
behavioral_data_drive

In [ ]:
task_data = pd.read_csv(os.path.join(behavioral_data_drive, '1b_S_NS_responses_per_subj.csv'))
task_data.set_index("subj_idx", inplace=True)
task_data.index.rename("Subject", inplace=True)
task_data.index = task_data.index.map(str)
print(task_data.shape)
task_data.head()

In [ ]:
task_data.iloc[0,:]

In [ ]:
task_data2 = pd.read_csv(os.path.join(behavioral_data_drive, '1e_S_NS_pc_uncertainResp_per_subj.csv'))
task_data2.set_index("subj_idx", inplace=True)
task_data2.index.rename("Subject", inplace=True)
task_data2.index = task_data2.index.map(str)
task_data2["pc_unc_total"] = (task_data2["pc_unc_Mental"] + task_data2["pc_unc_Rand"])/2
print(task_data2.shape)
task_data2.head()

In [ ]:
task_data = task_data.join(task_data2, how='inner')
print(task_data.shape)
task_data.head()

In [ ]:
data = task_data.join(behav_data, how='inner')
print(data.shape)
data.head()

In [ ]:
# Demographics for the paper
data.groupby(['Gender']).count()

In [ ]:
data['Age_in_Yrs'].describe()

Explore correlations between proportion of "social" responses and various trait phenotypes:

In [ ]:
def explore_correlation(x, y, data=data):
    """
    Calculates and plots correlation between x and y variables in dataframe `data`, plus distribution of x and y 
    """
    sns.set_style("white")
    
    inds = ~np.isnan(data[x]) & ~np.isnan(data[y]) # find rows where neither x or y is NaN

    g = sns.jointplot(x=x, y=y, data=data, kind='reg', color='gray')

    # Calculate and print correlations
    rp, pp = stats.pearsonr(data[x][inds], data[y][inds])
    rs, ps = stats.spearmanr(data[x][inds], data[y][inds])
    #g.ax_joint.annotate(f'r_s = {rs:.2f}\n(p={ps:.1e})', xy=(.05,.8), xycoords='axes fraction')
    print(f'Spearman r={rs}, p = {ps}')
    
    return g

In [ ]:
# The following function was taken from here: https://github.com/psinger/CorrelationStats/blob/master/corrstats.py

from scipy.stats import t, norm
from math import atanh, pow
from numpy import tanh

def rz_ci(r, n, conf_level = 0.95):
    zr_se = pow(1/(n - 3), .5)
    moe = norm.ppf(1 - (1 - conf_level)/float(2)) * zr_se
    zu = atanh(r) + moe
    zl = atanh(r) - moe
    return tanh((zl, zu))

def rho_rxy_rxz(rxy, rxz, ryz):
    num = (ryz-1/2.*rxy*rxz)*(1-pow(rxy,2)-pow(rxz,2)-pow(ryz,2))+pow(ryz,3)
    den = (1 - pow(rxy,2)) * (1 - pow(rxz,2))
    return num/float(den)

def dependent_corr(xy, xz, yz, n, twotailed=True, conf_level=0.95, method='steiger'):
    """
    Calculates the statistic significance between two dependent correlation coefficients
    @param xy: correlation coefficient between x and y
    @param xz: correlation coefficient between x and z
    @param yz: correlation coefficient between y and z
    @param n: number of elements in x, y and z
    @param twotailed: whether to calculate a one or two tailed test, only works for 'steiger' method
    @param conf_level: confidence level, only works for 'zou' method
    @param method: defines the method uses, 'steiger' or 'zou'
    @return: t and p-val
    """
    if method == 'steiger':
        d = xy - xz
        determin = 1 - xy * xy - xz * xz - yz * yz + 2 * xy * xz * yz
        av = (xy + xz)/2
        cube = (1 - yz) * (1 - yz) * (1 - yz)

        t2 = d * np.sqrt((n - 1) * (1 + yz)/(((2 * (n - 1)/(n - 3)) * determin + av * av * cube)))
        p = 1 - t.cdf(abs(t2), n - 3)

        if twotailed:
            p *= 2

        return t2, p
    elif method == 'zou':
        L1 = rz_ci(xy, n, conf_level=conf_level)[0]
        U1 = rz_ci(xy, n, conf_level=conf_level)[1]
        L2 = rz_ci(xz, n, conf_level=conf_level)[0]
        U2 = rz_ci(xz, n, conf_level=conf_level)[1]
        rho_r12_r13 = rho_rxy_rxz(xy, xz, yz)
        lower = xy - xz - pow((pow((xy - L1), 2) + pow((U2 - xz), 2) - 2 * rho_r12_r13 * (xy - L1) * (U2 - xz)), 0.5)
        upper = xy - xz + pow((pow((U1 - xy), 2) + pow((xz - L2), 2) - 2 * rho_r12_r13 * (U1 - xy) * (xz - L2)), 0.5)
        return lower, upper
    else:
        raise Exception('Wrong method!')





In [ ]:
data.shape, data.head()

## DV1: response % social - % nonsocial

In [ ]:
x = "Soc-NonSoc_pc"
y = "ASR_Intn_T"

g = explore_correlation(x, y)
g.ax_joint.set_xlabel('%"Social" - %"Non-social"')
g.ax_joint.set_ylabel('Internalizing score (T)')
#plt.text(0.1,10,r"$r_{S}=-.58$"f'\n(p='r"$.08^+)$")
g.ax_joint.set_xlim(-100,102)
g.ax_joint.annotate(r"$ r_{S}=.1$"f'\n(p='r"$.003)$", xy=(.02,.85), xycoords='axes fraction',fontsize=20) # 0.1, .003
g.savefig('../results/soc-nonsoc_vs_ASRIntnT.png')

In [ ]:
# externalization score
x = "Soc-NonSoc_pc"
y = "ASR_Extn_T"
g = explore_correlation(x, y)
g.ax_joint.set_ylabel('Externalizing score (T)')
g.ax_joint.set_xlabel('%"Social" - %"Non-social"')
g.ax_joint.set_xlim(-100,102)
#plt.text(0.1,10,r"$r_{S}=-.58$"f'\n(p='r"$.08^+)$")
g.ax_joint.annotate(r"$ r_{S}=.06$"f'\n(p='r"$.096)$", xy=(.02,.85), xycoords='axes fraction',fontsize=20) # 0.1, .003
g.savefig('../results/soc-nonsoc_vs_ASRExntT.png')

In [ ]:
x = "Soc-NonSoc_pc"
y = "ASR_Intn_T"
z = "ASR_Extn_T"

inds = ~np.isnan(data[x]) & ~np.isnan(data[y]) & ~np.isnan(data[z])
print(inds.sum())

rs_xy = stats.spearmanr(data[x][inds], data[y][inds])[0]
rs_xz = stats.spearmanr(data[x][inds], data[z][inds])[0]
rs_yz = stats.spearmanr(data[y][inds], data[z][inds])[0]
print(rs_xy)
print(rs_xz)
print(rs_yz)

Determine whether the difference in correlation between `Soc-NonSoc_pc` and internalizing vs externalizing symptoms is statistically significant:

In [ ]:
n = inds.sum()

xy = stats.spearmanr(data[x][inds], data[y][inds])[0]
xz = stats.spearmanr(data[x][inds], data[z][inds])[0]
yz = stats.spearmanr(data[y][inds], data[z][inds])[0]

dependent_corr(xy, xz, yz, n, twotailed=False)

## 2. RANDOM MECH

In [ ]:
def box_plot(data, edge_color, fill_color,pos,v):
    # customize boxplots
    alpha= .2
    data = data[~np.isnan(data)]
    bp = ax.boxplot(data, positions = [pos], patch_artist=True,widths=.6,vert=v,flierprops = dict(markeredgecolor=edge_color,
    markerfacecolor=fill_color, alpha=alpha))
    
    for element in ['boxes', 'whiskers', 'fliers', 'means', 'caps']:
        plt.setp(bp[element], color=edge_color)
    for element in ['medians']:
        plt.setp(bp[element], color='k',linewidth=2,ls='dashed')

    for patch in bp['boxes']:
        patch.set(facecolor=fill_color,alpha=alpha)#'w'       
        
    return bp

In [ ]:
#red_rgb = [103,0,31] # edges of RdBu
#blue_rgb = [5,48,97] # edges of RdBu
red_rgb =[188,61,62] # from Emily
blue_rgb = [54,122,177] # from Emily
red_rgb = np.array(red_rgb)/255
blue_rgb = np.array(blue_rgb)/255
alpha = .2 # transparency inside boxplots, for datapts etc.

#colors = [red_rgb,blue_rgb]
# Set your custom color palette
#myPalette = sns.set_palette(sns.color_palette(colors))

In [ ]:
#Look at responses on RANDMECH:
all_trial_data = pd.read_csv(os.path.join(behavioral_data_drive, 'hcp_social_826subs.csv'))
all_trial_data.set_index("subj_idx", inplace=True)
all_trial_data.index.rename("Subject", inplace=True)
all_trial_data.index = all_trial_data.index.map(str)
all_trial_data.head()

data = data.join(all_trial_data[all_trial_data["movie"]=="Random mechanical.AVI"]["response"])
data.rename(columns={"response": "rand_mech_response"}, inplace=True)
data.head()

In [ ]:
# Create a new variable denoting subjects who responded either "yes" or "unsure"
data["rand_mech_yesorunsure"] = data["rand_mech_response"] > 0
y = "ASR_Intn_T"

In [ ]:
#data1 = data.loc[data['rand_mech_response']!=9,:]
#data1.shape
fig, ax = plt.subplots(figsize=(5,6))
y = "ASR_Intn_T"
#x = "rand_mech_response"
x = "rand_mech_yesorunsure"
rows = data[x] == 0
bp2 = box_plot(data.loc[rows,y], blue_rgb, blue_rgb,0,True)
print(f'"Non-social": Mean int score: {np.nanmean(data.loc[rows,y])},SE: {stats.sem(data.loc[rows,y],nan_policy="omit")}')
rows = data[x] == 1
bp1 = box_plot(data.loc[rows,y], red_rgb, red_rgb,1,True)
print(f'"Social" or "Unsure": Mean int score: {np.nanmean(data.loc[rows,y])},SE: {stats.sem(data.loc[rows,y],nan_policy="omit")}')
plt.xticks(range(2),['"Non-social"','"Social"/\n "Unsure"'])
tscore, p = stats.ttest_ind(data[data[x]==1][y], data[data[x]==0][y], nan_policy='omit')
print(f't={tscore}, p={p}')
#plt.ylabel('ASR_Intn_T')
plt.ylim(30,104)
plt.plot([0,0,1,1],[96,98,98,96],linewidth = 1,color='k')
plt.plot([0.5],[99]*1,'*',color='k')

#ax.annotate(f't = {scoret:.2f}\n(p = {p:.2g})', xy=(.02,.85), xycoords='axes fraction')
plt.title('RANDOM MECH',fontweight='bold')
plt.ylabel('Internalizing score (T)')
#plt.ylim(0,100)
fig.savefig('../results/RANDMECHresp_vs_ASR_Intn_T.png',bbox_inches='tight')

In [ ]:
#data1 = data.loc[data['rand_mech_response']!=9,:]
#data1.shape
fig, ax = plt.subplots(figsize=(5,6))

y = "ASR_Extn_T"
#x = "rand_mech_response"
x = "rand_mech_yesorunsure"
rows = data[x] == 0
bp2 = box_plot(data.loc[rows,y], blue_rgb, blue_rgb,0,True)
print(f'"Non-social": Mean int score: {np.nanmean(data.loc[rows,y])},SE: {stats.sem(data.loc[rows,y],nan_policy="omit")}')
rows = data[x] == 1
bp1 = box_plot(data.loc[rows,y], red_rgb, red_rgb,1,True)
print(f'"Social" or "Unsure": Mean int score: {np.nanmean(data.loc[rows,y])},SE: {stats.sem(data.loc[rows,y],nan_policy="omit")}')
plt.xticks(range(2),['"Non-social"','"Social"/\n "Unsure"'])
tscore, p = stats.ttest_ind(data[data[x]==1][y], data[data[x]==0][y], nan_policy='omit')
print(f't={tscore}, p={p}')
#plt.ylabel('ASR_Extn_T')
plt.plot([0,0,1,1],[93,95,95,93],linewidth = 1,color='k')
plt.plot([0.5],[97]*1,'+',color='k')
   
#ax.annotate(f't = {tscore:.2f}\n(p = {p:.2g})', xy=(.02,.85), xycoords='axes fraction')
plt.title('RANDOM MECH',fontweight='bold')
plt.ylabel('Externalizing score (T)')
fig.savefig('../results/RANDMECHresp_vs_ASR_Extn_T.png',bbox_inches='tight')


(extra analysis we don't use) Look at RT on RANDMECH:

In [ ]:
data = data.join(all_trial_data[all_trial_data["movie"]=="Random mechanical.AVI"]["rt"])
data = data.rename(columns={"rt": "rand_mech_rt"})
data.head()

In [ ]:
g = explore_correlation(x="rand_mech_rt", y="ASR_Intn_T", data=data)
# no correlation between RT and internalizing score

In [ ]:
# Restrict correlation to only subjects that responded "yes" or "unsure"

g = explore_correlation(x="rand_mech_rt", y="ASR_Intn_T", data=data[data["rand_mech_yesorunsure"]==1])

# (c,d) "Unsure" responses for Mental and Random vs. ASR_Int 

In [ ]:
x = "pc_unc_total"
g = explore_correlation(x, y="ASR_Intn_T")
plt.xlabel('Mean(%Unsure"Social", %Unsure"Non-social")')
plt.xlim(0,100)
#g.savefig('../results/soc-nonsoc_vs_loneliness.png')

In [ ]:
x = "pc_unc_Rand"
g = explore_correlation(x, y="ASR_Intn_T")
plt.ylim(30,100)
plt.yticks(10*np.arange(3,11))
g.ax_joint.set_ylabel('Internalizing score (T)')#'Mean internalizing beh.\n (T-score, a.u.)')
g.ax_joint.annotate(r"$ r_{S}=.098$"f'\n(p='r"$.005)$", xy=(.02,.85), xycoords='axes fraction',fontsize=20) # 0.1, .003
g.ax_joint.set_xlabel('%"Unsure" Random')
g.ax_joint.set_xlim(-3,100)
g.savefig('../results/unsure_random_vs_asr_int.png')

x = "pc_unc_Mental"
g = explore_correlation(x, y="ASR_Intn_T")
plt.ylim(30,100)
plt.yticks(10*np.arange(3,11))
g.ax_joint.set_ylabel('Internalizing score (T)')# 'Mean internalizing beh.\n (T-score, a.u.)')
g.ax_joint.annotate(r"$ r_{S}=.024$"f'\n(p='r"$.49)$", xy=(.07,.85), xycoords='axes fraction',fontsize=20) # 0.1, .003
g.ax_joint.set_xlabel('%"Unsure" Mental')
g.ax_joint.set_xlim(-3,100)
g.savefig('../results/unsure_mental_vs_asr_int.png')

In [ ]:
x = 'ASR_Intn_T'
y = 'pc_unc_Rand'
z = 'pc_unc_Mental'

inds = ~np.isnan(data[x]) & ~np.isnan(data[y]) & ~np.isnan(data[z])

n = inds.sum()
xy = stats.spearmanr(data[x][inds], data[y][inds])[0]
xz = stats.spearmanr(data[x][inds], data[z][inds])[0]
yz = stats.spearmanr(data[y][inds], data[z][inds])[0]

dependent_corr(xy, xz, yz, n, twotailed=False)

In [ ]:
y = "ASR_Extn_T"
'''x = "pc_unc_total"
g = explore_correlation(x, y)
plt.xlabel('Mean(%Unsure"Social", %Unsure"Non-social")')
plt.xlim(0,100)'''
#g.savefig('../results/soc-nonsoc_vs_loneliness.png')

x = "pc_unc_Rand"
g = explore_correlation(x, y)
plt.xlabel('%Unsure Random')
plt.xlim(-1,100)
plt.ylim(30,100)
plt.yticks(10*np.arange(3,11))
plt.ylabel('Mean externalizing beh.\n (T-score, a.u.)')
g.savefig('../results/unsure_random_vs_asr_ext.png')

x = "pc_unc_Mental"
g = explore_correlation(x, y)
plt.xlabel('%Unsure Mental')
plt.xlim(-1,100)
plt.ylim(30,100)
plt.yticks(10*np.arange(3,11))
plt.ylabel('Mean externalizing beh.\n (T-score, a.u.)')
g.savefig('../results/unsure_mental_vs_asr_ext.png')
